In [ ]:
import pandas as pd
import pyodbc
import json 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [ ]:
 #Đọc file config
with open("etl_config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

# Chuỗi kết nối SQL Server
conn_str = (
    f"Driver={{{config['sqlserver']['odbc_driver']}}};"
    f"Server={config['sqlserver']['server']};"
    f"Database={config['sqlserver']['database']};"
    f"Trusted_Connection=yes;"
)

# Kết nối
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print(" Kết nối SQL Server thành công")


In [ ]:
import os
import pandas as pd

raw_folder = config["paths"]["raw_folder"]

# Lấy file Excel đầu tiên trong thư mục raw_data
raw_files = [f for f in os.listdir(raw_folder) if f.endswith(".xlsx")]
if not raw_files:
    raise FileNotFoundError("Không tìm thấy file Excel trong raw_data")

file_path = os.path.join(raw_folder, raw_files[0])
print(" Đang đọc file:", file_path)

# Đọc file Excel
df = pd.read_excel(file_path)
display(df.head())



In [ ]:
# Bỏ các cột không tên
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]


In [ ]:
df = df.dropna(how="all")


In [ ]:
df.columns = df.columns.str.strip()       # bỏ khoảng trắng thừa
df.columns = df.columns.str.replace(" ", "", regex=False)  
df.columns = df.columns.str.replace("-", "_", regex=False)  


In [ ]:
# Chuyển NgayMua về datetime chuẩn
df["NgayMua"] = pd.to_datetime(df["NgayMua"], errors="coerce", dayfirst=True)

# Nếu có dòng nào không convert được -> sẽ thành NaT
print("Số dòng null ở NgayMua sau khi chuyển:", df["NgayMua"].isna().sum())


In [ ]:
print("Dữ liệu sau khi làm sạch:")
display(df.head())
print("Các cột còn lại:", df.columns.tolist())


In [ ]:
cursor.execute("""
IF OBJECT_ID('dbo.Staging_SaleData', 'U') IS NULL
BEGIN
    CREATE TABLE Staging_SaleData (
        HoTen NVARCHAR(255),
        Email NVARCHAR(255),
        SDT VARCHAR(20),
        GioiTinh NVARCHAR(10),
        NgaySinh DATE,
        DonHang NVARCHAR(50),
        NgayMua DATE,
        Traffic VARCHAR(100),
        TinhThanh NVARCHAR(100),
        QuanHuyen NVARCHAR(100),
        TenSanPham NVARCHAR(255),
        TenNhomSanPham NVARCHAR(100),
        SKU NVARCHAR(50),
        PhienBan NVARCHAR(100),
        NhaSanXuat NVARCHAR(100),
        PhuongThucTT NVARCHAR(100),
        SoLuong INT,
        DoanhThu FLOAT,
        TienKhuyenMai FLOAT,
        VanChuyen FLOAT,
        DoanhThuThuan FLOAT
    )
END
""")
conn.commit()
print(" Đã tạo bảng Staging_SaleData ")



In [ ]:
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Staging_SaleData (
            HoTen, Email, SDT, GioiTinh, NgaySinh,
            DonHang, NgayMua, Traffic, TinhThanh, QuanHuyen,
            TenSanPham, TenNhomSanPham, SKU, PhienBan, NhaSanXuat,
            PhuongThucTT, SoLuong, DoanhThu, TienKhuyenMai,
            VanChuyen, DoanhThuThuan
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row.HoTen,
        row.Email,
        row.SDT,
        row.GioiTinh,
        # Convert ngày sinh
        pd.to_datetime(row.NgaySinh, errors="coerce").strftime("%Y-%m-%d") if pd.notnull(row.NgaySinh) else None,
        row.DonHang,
        # Convert ngày mua
        pd.to_datetime(row.NgayMua, errors="coerce").strftime("%Y-%m-%d") if pd.notnull(row.NgayMua) else None,
        row.Traffic,
        row.TinhThanh,
        row.QuanHuyen,
        row.TenSanPham,
        row.TenNhomSanPham,
        row.SKU,
        row.PhienBan,
        row.NhaSanXuat,
        row.PhuongThucTT,
        int(row.SoLuong),
        float(row.DoanhThu),
        float(row.TienKhuyenMai),
        float(row.VanChuyen),
        float(row.DoanhThuThuan)
    ))


In [ ]:
cursor.execute("""
IF OBJECT_ID('dbo.Dim_KhachHang', 'U') IS NULL
BEGIN
    CREATE TABLE Dim_KhachHang (
        KhachHangID INT IDENTITY(1,1) PRIMARY KEY,
        HoTen NVARCHAR(255),
        Email NVARCHAR(255),
        SDT VARCHAR(20),
        GioiTinh NVARCHAR(10),
        NgaySinh DATE,
        TinhThanh NVARCHAR(100),
        QuanHuyen NVARCHAR(100),
        Traffic NVARCHAR(100)
    )
END
""")
conn.commit()
print("Đã tạo Dim_KhachHang ")


In [ ]:
cursor.execute("""
IF OBJECT_ID('dbo.Dim_NhomSP', 'U') IS NULL
BEGIN
    CREATE TABLE Dim_NhomSP (
        NhomSPID INT IDENTITY(1,1) PRIMARY KEY,
        TenNhomSanPham NVARCHAR(100)
    )
END
""")
conn.commit()
print(" Đã tạo Dim_NhomSP")


In [ ]:
cursor.execute("""
IF OBJECT_ID('dbo.Dim_SP', 'U') IS NULL
BEGIN
    CREATE TABLE Dim_SP (
        SPID INT IDENTITY(1,1) PRIMARY KEY,   -- Khóa chính kỹ thuật
        SKU NVARCHAR(50) NOT NULL,            -- SKU là mã duy nhất
        TenSanPham NVARCHAR(255),
        NhaSanXuat NVARCHAR(100),
        PhienBan NVARCHAR(100),
        CONSTRAINT UQ_DimSP_SKU UNIQUE (SKU)  -- Đảm bảo không trùng SKU
    )
END
""")
conn.commit()
print(" Đã tạo Dim_SP với UNIQUE(SKU)")



In [ ]:
# Xóa các dòng trùng SKU, chỉ giữ lại 1 dòng duy nhất (SPID nhỏ nhất)
cursor.execute("""
;WITH CTE AS (
    SELECT 
        SPID,
        SKU,
        ROW_NUMBER() OVER (PARTITION BY SKU ORDER BY SPID) AS rn
    FROM Dim_SP
)
DELETE FROM CTE WHERE rn > 1;
""")
conn.commit()
print("Đã xoá SKU trùng, chỉ giữ lại 1 bản ghi duy nhất cho mỗi SKU")


In [ ]:
cursor.execute("""
IF OBJECT_ID('dbo.Dim_Date', 'U') IS NULL
BEGIN
    CREATE TABLE Dim_Date (
        DateID INT IDENTITY(1,1) PRIMARY KEY,
        Ngay DATE,
        Thang INT,
        Quy INT,
        Nam INT
    )
END
""")
conn.commit()
print(" Đã tạo Dim_Date")


In [ ]:
cursor.execute("""
IF OBJECT_ID('dbo.Fact_DonHang', 'U') IS NULL
BEGIN
    CREATE TABLE Fact_DonHang (
        DonHang NVARCHAR(50),   -- Mã đơn hàng gốc
        KhachHangID INT,
        SPID INT,
        NhomSPID INT,
        DateID INT,
        SoLuong INT,
        DoanhThu FLOAT,
        TienKhuyenMai FLOAT,
        DoanhThuThuan FLOAT,
        VanChuyen FLOAT,
        PRIMARY KEY (DonHang, SPID),   -- Khóa chính kép
        FOREIGN KEY (KhachHangID) REFERENCES Dim_KhachHang(KhachHangID),
        FOREIGN KEY (SPID) REFERENCES Dim_SP(SPID),
        FOREIGN KEY (NhomSPID) REFERENCES Dim_NhomSP(NhomSPID),
        FOREIGN KEY (DateID) REFERENCES Dim_Date(DateID)
    )
END
""")
conn.commit()
print("Đã tạo Fact_DonHang với PK (DonHang, SPID)")


In [ ]:
cursor.execute("""
;WITH LatestCustomer AS (
    SELECT 
        s.Email,
        s.HoTen,
        s.SDT,
        s.GioiTinh,
        s.NgaySinh,
        s.TinhThanh,
        s.QuanHuyen,
        s.Traffic,
        ROW_NUMBER() OVER (PARTITION BY s.Email ORDER BY s.NgayMua DESC) AS rn
    FROM Staging_SaleData s
)
INSERT INTO Dim_KhachHang (HoTen, Email, SDT, GioiTinh, NgaySinh, TinhThanh, QuanHuyen, Traffic)
SELECT HoTen, Email, SDT, GioiTinh, NgaySinh, TinhThanh, QuanHuyen, Traffic
FROM LatestCustomer lc
WHERE rn = 1
  AND NOT EXISTS (
      SELECT 1 FROM Dim_KhachHang kh WHERE kh.Email = lc.Email
  );
""")
conn.commit()
print(" Đã nạp dữ liệu vào Dim_KhachHang")


In [ ]:
cursor.execute("""
;WITH CTE AS (
    SELECT 
        s.SKU,
        s.TenSanPham,
        s.NhaSanXuat,
        s.PhienBan,
        ROW_NUMBER() OVER (PARTITION BY s.SKU ORDER BY s.TenSanPham) AS rn
    FROM Staging_SaleData s
    WHERE s.SKU IS NOT NULL
)
INSERT INTO Dim_SP (SKU, TenSanPham, NhaSanXuat, PhienBan)
SELECT SKU, TenSanPham, NhaSanXuat, PhienBan
FROM CTE
WHERE rn = 1;
""")
conn.commit()
print(" Đã nạp dữ liệu vào Dim_SP (mỗi SKU 1 bản duy nhất)")

In [ ]:
cursor.execute("""
INSERT INTO Dim_NhomSP (TenNhomSanPham)
SELECT DISTINCT s.TenNhomSanPham
FROM Staging_SaleData s
WHERE NOT EXISTS (
    SELECT 1 FROM Dim_NhomSP ns WHERE ns.TenNhomSanPham = s.TenNhomSanPham
)
""")
conn.commit()
print(" Đã nạp dữ liệu vào Dim_NhomSP")


In [ ]:
cursor.execute("""
INSERT INTO Dim_Date (Ngay)
SELECT DISTINCT s.NgayMua
FROM Staging_SaleData s
WHERE NOT EXISTS (
    SELECT 1 FROM Dim_Date d WHERE d.Ngay = s.NgayMua
)
""")
conn.commit()
print(" Đã nạp dữ liệu vào Dim_Date")


In [ ]:
cursor.execute("""
UPDATE Dim_Date
SET 
    Thang = MONTH(Ngay),
    Quy   = DATEPART(QUARTER, Ngay),
    Nam   = YEAR(Ngay)
WHERE Thang IS NULL OR Quy IS NULL OR Nam IS NULL
""")
conn.commit()
print(" Đã cập nhật Thang/Quy/Nam cho Dim_Date")


In [ ]:
cursor.execute("""
;WITH CTE AS (
    SELECT 
        sa.DonHang,
        kh.KhachHangID,
        sp.SPID,
        ns.NhomSPID,
        d.DateID,
        sa.SoLuong,
        sa.DoanhThu,
        sa.TienKhuyenMai,
        sa.DoanhThuThuan,
        sa.VanChuyen,
        ROW_NUMBER() OVER (PARTITION BY sa.DonHang, sp.SPID ORDER BY d.DateID DESC) AS rn
    FROM Staging_SaleData sa
    JOIN Dim_KhachHang kh ON sa.Email = kh.Email
    JOIN Dim_SP sp ON sa.SKU = sp.SKU
    JOIN Dim_NhomSP ns ON sa.TenNhomSanPham = ns.TenNhomSanPham
    JOIN Dim_Date d ON sa.NgayMua = d.Ngay
)
MERGE Fact_DonHang AS target
USING (
    SELECT * FROM CTE WHERE rn = 1
) AS src
ON (target.DonHang = src.DonHang AND target.SPID = src.SPID)
WHEN MATCHED THEN
    UPDATE SET 
        target.SoLuong = src.SoLuong,
        target.DoanhThu = src.DoanhThu,
        target.TienKhuyenMai = src.TienKhuyenMai,
        target.DoanhThuThuan = src.DoanhThuThuan,
        target.VanChuyen = src.VanChuyen
WHEN NOT MATCHED THEN
    INSERT (DonHang, KhachHangID, SPID, NhomSPID, DateID,
            SoLuong, DoanhThu, TienKhuyenMai, DoanhThuThuan, VanChuyen)
    VALUES (src.DonHang, src.KhachHangID, src.SPID, src.NhomSPID, src.DateID,
            src.SoLuong, src.DoanhThu, src.TienKhuyenMai, src.DoanhThuThuan, src.VanChuyen);
""")

conn.commit()
print("✅ Đã MERGE dữ liệu (giữ bản mới nhất) vào Fact_DonHang")


Tính RFM và KMean 


# Chuẩn hóa dữ liệu trước khi dùng Kmean


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# ==========================================
# Bước 1-2: Lấy dữ liệu từ database
# ==========================================
query = """
SELECT
    f.KhachHangID,
    f.DonHang,
    d.Ngay,
    f.DoanhThuThuan
FROM Fact_DonHang f
JOIN Dim_Date d ON f.DateID = d.DateID
"""
df = pd.read_sql(query, conn)

# Xử lý dữ liệu ngày
df["Ngay"] = pd.to_datetime(df["Ngay"], errors="coerce").fillna(pd.Timestamp("2021-04-04"))

# ==========================================
# Bước 3: Tính RFM
# ==========================================
ngay_tham_chieu = df["Ngay"].max() + pd.Timedelta(days=1)
print(f"📅 Ngày tham chiếu: {ngay_tham_chieu}")

df_rfm = df.groupby("KhachHangID").agg(
    NgayMuaGanNhat=("Ngay", "max"),          
    SoLanMua=("DonHang", "nunique"),        
    TongTien=("DoanhThuThuan", "sum")
).reset_index()

df_rfm["Recency"] = (ngay_tham_chieu - df_rfm["NgayMuaGanNhat"]).dt.days
df_rfm["Frequency"] = df_rfm["SoLanMua"]
df_rfm["Monetary"] = df_rfm["TongTien"]

df_rfm_final = df_rfm[["KhachHangID", "Recency", "Frequency", "Monetary"]].copy()

# ==========================================
# Bước 4: Lưu RFM vào database
# ==========================================
cursor.execute("""
IF OBJECT_ID('RFM_KhachHang', 'U') IS NULL
BEGIN
    CREATE TABLE RFM_KhachHang (
        KhachHangID INT PRIMARY KEY,
        Recency INT,
        Frequency FLOAT,
        Monetary FLOAT
    );
END
""")
conn.commit()

for _, row in df_rfm_final.iterrows():
    cursor.execute("""
        MERGE RFM_KhachHang AS target
        USING (SELECT ? AS KhachHangID, ? AS Recency, ? AS Frequency, ? AS Monetary) AS source
        ON target.KhachHangID = source.KhachHangID
        WHEN MATCHED THEN
            UPDATE SET Recency = source.Recency,
                       Frequency = source.Frequency,
                       Monetary = source.Monetary
        WHEN NOT MATCHED THEN
            INSERT (KhachHangID, Recency, Frequency, Monetary)
            VALUES (source.KhachHangID, source.Recency, source.Frequency, source.Monetary);
    """, row.KhachHangID, int(row.Recency), float(row.Frequency), float(row.Monetary))
conn.commit()
print("✅ Đã cập nhật bảng RFM_KhachHang!")

# ==========================================
# Bước 5: Chuẩn hóa dữ liệu
# ==========================================
X = df_rfm_final[["Recency", "Frequency", "Monetary"]]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ==========================================
# Bước 6: Tìm số cụm tối ưu (Elbow + Silhouette)
# ==========================================
sse = []
sil_scores = []
K_range = range(2, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    sse.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Vẽ biểu đồ
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(K_range, sse, marker="o")
plt.xlabel("Số cụm K")
plt.ylabel("SSE")
plt.title("Elbow Method")

plt.subplot(1, 2, 2)
plt.plot(K_range, sil_scores, marker="o", color="orange")
plt.xlabel("Số cụm K")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score")
plt.tight_layout()
plt.show()

# In silhouette score
print("\n📊 Silhouette Score cho từng K:")
for k, score in zip(K_range, sil_scores):
    print(f"K={k}, Silhouette Score={score:.3f}")

# ==========================================
# TỰ ĐỘNG TÌM ĐIỂM KHUỶU TAY - PHƯƠNG PHÁP KNEEDLE
# ==========================================
def tim_diem_khuyu_tay_kneedle(K_range, sse):
    """
    Thuật toán Kneedle: Tìm điểm có khoảng cách lớn nhất từ đường thẳng
    nối điểm đầu và điểm cuối
    
    Logic:
    1. Vẽ đường thẳng từ điểm đầu (K=2) đến điểm cuối (K=9)
    2. Tính khoảng cách từ mỗi điểm đến đường thẳng đó
    3. Điểm có khoảng cách xa nhất = điểm "cong" nhất = điểm khuỷu tay
    """
    # Chuẩn hóa dữ liệu về [0, 1] để tránh ảnh hưởng của đơn vị đo
    k_norm = (np.array(K_range) - min(K_range)) / (max(K_range) - min(K_range))
    sse_norm = (np.array(sse) - min(sse)) / (max(sse) - min(sse))
    
    # Tính khoảng cách từ mỗi điểm đến đường thẳng nối 2 đầu
    # Công thức khoảng cách từ điểm đến đường thẳng: d = |ax + by + c| / sqrt(a^2 + b^2)
    x1, y1 = k_norm[0], sse_norm[0]      # Điểm đầu
    x2, y2 = k_norm[-1], sse_norm[-1]    # Điểm cuối
    
    khoang_cach = []
    for i in range(len(k_norm)):
        x0, y0 = k_norm[i], sse_norm[i]
        # Khoảng cách từ điểm (x0, y0) đến đường thẳng
        d = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1) / np.sqrt((y2-y1)**2 + (x2-x1)**2)
        khoang_cach.append(d)
    
    # Điểm có khoảng cách lớn nhất là điểm khuỷu tay
    elbow_idx = np.argmax(khoang_cach)
    return K_range[elbow_idx], khoang_cach

# Tìm điểm khuỷu tay
optimal_k, distances = tim_diem_khuyu_tay_kneedle(K_range, sse)

print("\n" + "="*60)
print("🔍 TÌM K TỐI ƯU - PHƯƠNG PHÁP KNEEDLE (ELBOW)")
print("="*60)
print(f"✅ K tối ưu được phát hiện: {optimal_k}")
print(f"📊 Khoảng cách các điểm đến đường thẳng:")
for k, dist in zip(K_range, distances):
    marker = " ← ĐIỂM KHUỶU TAY" if k == optimal_k else ""
    print(f"   K={k}: {dist:.4f}{marker}")
print("="*60)

# Vẽ biểu đồ Elbow với điểm khuỷu tay được đánh dấu
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(K_range, sse, marker="o", linewidth=2, markersize=8)
# Vẽ đường thẳng nối 2 đầu
plt.plot([K_range[0], K_range[-1]], [sse[0], sse[-1]], 
         'r--', linewidth=1, alpha=0.5, label='Đường thẳng tham chiếu')
# Đánh dấu điểm khuỷu tay
plt.scatter([optimal_k], [sse[list(K_range).index(optimal_k)]], 
            color='red', s=300, zorder=5, marker='*', 
            label=f'Điểm khuỷu tay: K={optimal_k}', edgecolors='black', linewidths=2)
plt.xlabel("Số cụm K", fontsize=12)
plt.ylabel("SSE (Sum of Squared Errors)", fontsize=12)
plt.title("Elbow Method - Phương pháp Kneedle", fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.bar(K_range, distances, color='skyblue', edgecolor='black')
plt.bar([optimal_k], [distances[list(K_range).index(optimal_k)]], 
        color='red', edgecolor='black', label=f'K tối ưu = {optimal_k}')
plt.xlabel("Số cụm K", fontsize=12)
plt.ylabel("Khoảng cách đến đường thẳng", fontsize=12)
plt.title("Khoảng cách từ các điểm đến đường thẳng", fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\n✅ HỆ THỐNG TỰ ĐỘNG CHỌN K = {optimal_k}")
print(f"   💡 Nếu muốn thay đổi, hãy bỏ comment dòng bên dưới và chọn K thủ công")

# Hoặc bạn có thể tự chọn K thủ công
# optimal_k = 4  # Bỏ comment dòng này nếu muốn tự chọn

# ==========================================
# Bước 7: Chạy K-Means với K đã chọn
# ==========================================
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_rfm_final["Cluster"] = kmeans.fit_predict(X_scaled)

print(f"\n📊 Đã phân thành {optimal_k} cụm")
print(df_rfm_final.head(10))

# ==========================================
# Bước 8: TỰ ĐỘNG GÁN NHÃN PHÂN KHÚC
# ==========================================
# Phân tích đặc điểm từng cụm
cluster_summary = df_rfm_final.groupby("Cluster")[["Recency", "Frequency", "Monetary"]].mean()
print("\n📌 Đặc điểm trung bình từng cụm:")
print(cluster_summary.round(2))

# Tính điểm RFM chuẩn hóa cho từng cụm (để so sánh)
# Recency: càng thấp càng tốt → đảo ngược
# Frequency: càng cao càng tốt
# Monetary: càng cao càng tốt
r_max = cluster_summary['Recency'].max()
f_max = cluster_summary['Frequency'].max()
m_max = cluster_summary['Monetary'].max()

cluster_summary['R_Score'] = 1 - (cluster_summary['Recency'] / r_max)  # Đảo ngược
cluster_summary['F_Score'] = cluster_summary['Frequency'] / f_max
cluster_summary['M_Score'] = cluster_summary['Monetary'] / m_max

# Tổng điểm RFM
cluster_summary['RFM_Score'] = (
    cluster_summary['R_Score'] + 
    cluster_summary['F_Score'] + 
    cluster_summary['M_Score']
) / 3

print("\n📊 Điểm RFM chuẩn hóa (0-1) cho từng cụm:")
print(cluster_summary[['R_Score', 'F_Score', 'M_Score', 'RFM_Score']].round(3))

# Sắp xếp cụm theo điểm RFM từ cao đến thấp
cluster_ranking = cluster_summary.sort_values('RFM_Score', ascending=False)
print("\n🏆 Xếp hạng cụm (từ tốt nhất đến kém nhất):")
print(cluster_ranking[['Recency', 'Frequency', 'Monetary', 'RFM_Score']].round(2))

# ==========================================
# HÀM TỰ ĐỘNG GÁN TÊN PHÂN KHÚC
# ==========================================
def tu_dong_gan_phan_khuc(optimal_k, cluster_ranking):
    """
    Tự động gán tên phân khúc dựa trên số cụm K và xếp hạng RFM
    
    Logic:
    - Cụm có RFM_Score cao nhất → Phân khúc tốt nhất
    - Cụm có RFM_Score thấp nhất → Phân khúc kém nhất
    - Các cụm ở giữa → Phân khúc trung gian
    """
    phan_khuc_mapping = {}
    
    # Danh sách tên phân khúc từ tốt đến kém (có thể tùy chỉnh)
    if optimal_k == 2:
        ten_phan_khuc = [
            "Khách hàng tốt",
            "Khách hàng yếu"
        ]
    elif optimal_k == 3:
        ten_phan_khuc = [
            "Khách hàng VIP",
            "Khách hàng trung bình",
            "Khách hàng yếu"
        ]
    elif optimal_k == 4:
        ten_phan_khuc = [
            "Khách hàng VIP",
            "Khách hàng trung thành",
            "Khách hàng tiềm năng",
            "Khách hàng có nguy cơ mất"
        ]
    elif optimal_k == 5:
        ten_phan_khuc = [
            "Khách hàng VIP",
            "Khách hàng trung thành",
            "Khách hàng ổn định",
            "Khách hàng cần chăm sóc",
            "Khách hàng có nguy cơ mất"
        ]
    elif optimal_k == 6:
        ten_phan_khuc = [
            "Khách hàng VIP",
            "Khách hàng trung thành",
            "Khách hàng tiềm năng",
            "Khách hàng ổn định",
            "Khách hàng cần chăm sóc",
            "Khách hàng có nguy cơ mất"
        ]
    else:  # K >= 7
        # Tạo tên động cho số cụm lớn
        ten_phan_khuc = [f"Phân khúc {i+1} (Điểm cao)" if i == 0 
                        else f"Phân khúc {i+1} (Điểm thấp)" if i == optimal_k-1
                        else f"Phân khúc {i+1}" 
                        for i in range(optimal_k)]
    
    # Gán tên cho từng cluster theo thứ tự xếp hạng
    for idx, (cluster_id, row) in enumerate(cluster_ranking.iterrows()):
        phan_khuc_mapping[cluster_id] = ten_phan_khuc[idx]
    
    return phan_khuc_mapping

# Tạo mapping tự động
phan_khuc_mapping = tu_dong_gan_phan_khuc(optimal_k, cluster_ranking)

print("\n🏷️ Ánh xạ Cluster → Phân khúc:")
for cluster_id, ten_phan_khuc in sorted(phan_khuc_mapping.items()):
    print(f"  Cluster {cluster_id} → {ten_phan_khuc}")

# Áp dụng gán nhãn
df_rfm_final["PhanKhuc"] = df_rfm_final["Cluster"].map(phan_khuc_mapping)

# Hiển thị kết quả
print("\n📊 Phân bổ khách hàng theo phân khúc:")
print(df_rfm_final["PhanKhuc"].value_counts().sort_index())

print("\n📋 Mẫu dữ liệu với phân khúc:")
print(df_rfm_final.head(20))

# ==========================================
# Bước 9: Lưu kết quả vào database
# ==========================================
cursor.execute("""
IF OBJECT_ID('Customer_Segmentation', 'U') IS NULL
BEGIN
    CREATE TABLE Customer_Segmentation (
        KhachHangID INT PRIMARY KEY,
        Recency INT,
        Frequency FLOAT,
        Monetary FLOAT,
        Cluster INT,
        PhanKhuc NVARCHAR(100)
    );
END
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM sys.columns 
                   WHERE object_id = OBJECT_ID('Customer_Segmentation') 
                   AND name = 'PhanKhuc')
    BEGIN
        ALTER TABLE Customer_Segmentation ADD PhanKhuc NVARCHAR(100);
    END
END
""")
conn.commit()

for _, row in df_rfm_final.iterrows():
    cursor.execute("""
        MERGE Customer_Segmentation AS target
        USING (SELECT ? AS KhachHangID, ? AS Recency, ? AS Frequency, 
                      ? AS Monetary, ? AS Cluster, ? AS PhanKhuc) AS source
        ON target.KhachHangID = source.KhachHangID
        WHEN MATCHED THEN
            UPDATE SET Recency = source.Recency,
                       Frequency = source.Frequency,
                       Monetary = source.Monetary,
                       Cluster = source.Cluster,
                       PhanKhuc = source.PhanKhuc
        WHEN NOT MATCHED THEN
            INSERT (KhachHangID, Recency, Frequency, Monetary, Cluster, PhanKhuc)
            VALUES (source.KhachHangID, source.Recency, source.Frequency, 
                    source.Monetary, source.Cluster, source.PhanKhuc);
    """, row.KhachHangID, int(row.Recency), float(row.Frequency), 
        float(row.Monetary), int(row.Cluster), row.PhanKhuc)

conn.commit()
print("\n✅ Đã lưu kết quả phân cụm và phân khúc vào database!")

# ==========================================
# Bước 10: Visualization
# ==========================================
import seaborn as sns

plt.figure(figsize=(14, 5))

# Biểu đồ 1: Phân bổ phân khúc
plt.subplot(1, 3, 1)
phan_khuc_counts = df_rfm_final["PhanKhuc"].value_counts()
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(phan_khuc_counts)))
phan_khuc_counts.plot(kind="bar", color=colors)
plt.title(f"Phân bổ {optimal_k} phân khúc khách hàng")
plt.xlabel("Phân khúc")
plt.ylabel("Số lượng khách hàng")
plt.xticks(rotation=45, ha='right')

# Biểu đồ 2: Recency vs Monetary
plt.subplot(1, 3, 2)
for segment in df_rfm_final["PhanKhuc"].unique():
    data = df_rfm_final[df_rfm_final["PhanKhuc"] == segment]
    plt.scatter(data["Recency"], data["Monetary"], label=segment, alpha=0.6)
plt.xlabel("Recency (ngày)")
plt.ylabel("Monetary (VND)")
plt.title("Recency vs Monetary")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# Biểu đồ 3: Frequency vs Monetary
plt.subplot(1, 3, 3)
for segment in df_rfm_final["PhanKhuc"].unique():
    data = df_rfm_final[df_rfm_final["PhanKhuc"] == segment]
    plt.scatter(data["Frequency"], data["Monetary"], label=segment, alpha=0.6)
plt.xlabel("Frequency (lần mua)")
plt.ylabel("Monetary (VND)")
plt.title("Frequency vs Monetary")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

plt.tight_layout()
plt.show()

# In bảng tổng kết cuối cùng
print("\n" + "="*60)
print("📊 TỔNG KẾT PHÂN KHÚC KHÁCH HÀNG")
print("="*60)
final_summary = df_rfm_final.groupby("PhanKhuc").agg(
    SoLuong=("KhachHangID", "count"),
    R_TrungBinh=("Recency", "mean"),
    F_TrungBinh=("Frequency", "mean"),
    M_TrungBinh=("Monetary", "mean")
).round(2)
print(final_summary)
print("="*60)